### Loading Libraries in Python

In [53]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import math
import random as r
%matplotlib inline

### A Basic LIF Neuron 

In [54]:
class LIFNeuron(object):
    
    def __init__(self, u_rest=0.0, u_thresh=1.0, tau_rest=4.0, r=1.0, tau=10.0):
        
        # Membrane resting potential in mV
        self.u_rest = u_rest
        # Membrane threshold potential in mV
        self.u_thresh = u_thresh
        # Duration of the resting period in ms
        self.tau_rest = tau_rest
        # Membrane resistance in Ohm
        self.r = r
        # Membrane time constant in ms
        self.tau = tau
        
        # Instantiate a graph for this neuron
        self.graph = tf.Graph()
        
        # Build the graph
        with self.graph.as_default():
        
            # Variables and placeholders
            self.get_vars_and_ph()
            
            # Operations
            self.input = self.get_input_op()
            self.potential = self.get_potential_op()
            # Note that input is a prerequisite of potential, so it will
            # always be evaluated when potential is
            
    # Variables and placeholders
    def get_vars_and_ph(self):

        # The current membrane potential
        self.u = tf.Variable(self.u_rest, dtype=tf.float32, name='u')
        # The duration left in the resting period (0 most of the time except after a neuron spike)
        self.t_rest = tf.Variable(0.0, dtype=tf.float32, name='t_rest')
        # Input current
        self.i_app = tf.placeholder(dtype=tf.float32, name='i_app')
        # The chosen time interval for the stimulation in ms
        self.dt = tf.placeholder(dtype=tf.float32, name='dt')

    # Evaluate input current
    def get_input_op(self):
        
        return self.i_app
        
    # Neuron behaviour during integration phase (below threshold)
    def get_integrating_op(self):

        # Get input current
        i_op = self.get_input_op()

        # Update membrane potential
        du_op = tf.divide(tf.subtract(tf.multiply(self.r, i_op), self.u), self.tau) 
        u_op = self.u.assign_add(du_op * self.dt)
        # Refractory period is 0
        t_rest_op = self.t_rest.assign(0.0)
        
        return u_op, t_rest_op

    # Neuron behaviour during firing phase (above threshold)    
    def get_firing_op(self):                  

        # Reset membrane potential
        u_op = self.u.assign(self.u_rest)
        # Refractory period starts now
        t_rest_op = self.t_rest.assign(self.tau_rest)

        return u_op, t_rest_op

    # Neuron behaviour during resting phase (t_rest > 0)
    def get_resting_op(self):

        # Membrane potential stays at u_rest
        u_op = self.u.assign(self.u_rest)
        # Refractory period is decreased by dt
        t_rest_op = self.t_rest.assign_sub(self.dt)
        
        return u_op, t_rest_op

    def get_potential_op(self):
        
        return tf.case(
            [
                (self.t_rest > 0.0, self.get_resting_op),
                (self.u > self.u_thresh, self.get_firing_op),
            ],
            default=self.get_integrating_op
        )

### A new neuron model derived from the LIF neuron

In [55]:
# It takes synaptic spikes as input and remember them over a specified time period
class LIFSynapticNeuron(LIFNeuron):
    
    def __init__(self, n_syn, w, max_spikes=50, u_rest=0.0, u_thresh=1.0, tau_rest=4.0, r=1.0, tau=10.0, q=1.5, tau_syn=5.0):
      
        # Number of synapses
        self.n_syn = n_syn
        # Maximum number of spikes we remember
        self.max_spikes = max_spikes
        # The neuron synaptic 'charge'
        self.q = q
        # The synaptic time constant (ms)
        self.tau_syn = tau_syn
        # The synaptic efficacy
        self.w = w

        super(LIFSynapticNeuron, self).__init__(u_rest, u_thresh, tau_rest, r, tau)
    
    # Update the parent graph variables and placeholders
    def get_vars_and_ph(self):
        
        # Get parent grah variables and placeholders
        super(LIFSynapticNeuron, self).get_vars_and_ph()

        # Add ours
        
        # The history of synaptic spike times for the neuron 
        self.t_spikes = tf.Variable(tf.constant(-1.0, shape=[self.max_spikes, self.n_syn], dtype=tf.float32))
        # The last index used to insert spike times
        self.t_spikes_idx = tf.Variable(self.max_spikes-1, dtype=tf.int32)
        # A placeholder indicating which synapse spiked in the last time step
        self.syn_has_spiked = tf.placeholder(shape=[self.n_syn], dtype=tf.bool)

    # Operation to update spike times
    def update_spike_times(self):
        
        # Increase the age of older spikes
        old_spikes_op = self.t_spikes.assign_add(tf.where(self.t_spikes >=0,
                                                          tf.constant(1.0, shape=[self.max_spikes, self.n_syn]) * self.dt,
                                                          tf.zeros([self.max_spikes, self.n_syn])))

        # Increment last spike index (modulo max_spikes)
        new_idx_op = self.t_spikes_idx.assign(tf.mod(self.t_spikes_idx + 1, self.max_spikes))

        # Create a list of coordinates to insert the new spikes
        idx_op = tf.constant(1, shape=[self.n_syn], dtype=tf.int32) * new_idx_op
        coord_op = tf.stack([idx_op, tf.range(self.n_syn)], axis=1)

        # Create a vector of new spike times (non-spikes are assigned a negative time)
        new_spikes_op = tf.where(self.syn_has_spiked,
                                 tf.constant(0.0, shape=[self.n_syn]),
                                 tf.constant(-1.0, shape=[self.n_syn]))
        
        # Replace older spikes by new ones
        return tf.scatter_nd_update(old_spikes_op, coord_op, new_spikes_op)

    # Override parent get_input_op method
    def get_input_op(self):
        
        # Update our memory of spike times with the new spikes
        t_spikes_op = self.update_spike_times()

        # Evaluate synaptic input current for each spike on each synapse
        i_syn_op = tf.where(t_spikes_op >=0,
                            self.q/self.tau_syn * tf.exp(tf.negative(t_spikes_op/self.tau_syn)),
                            t_spikes_op*0.0)

        # Add each synaptic current to the input current
        i_op =  tf.reduce_sum(self.w * i_syn_op)
        
        return tf.add(self.i_app, i_op)

### LIF Neuron for Synaptic Inputs

In [56]:
class LIFNeuron(object):

    def __init__(self,
                 n_syn, w, max_spikes=None, 
                 p_rest=0.0, tau_rest=1.0, tau_m=10.0, tau_s=2.5, T=None,
                 K=2.1, K1=2.0, K2=4.0):

        # Model parameters

        # Membrane resting potential
        self.p_rest = p_rest
        
        # Duration of the recovery period
        self.tau_rest = tau_rest
        
        # Membrane time constant
        self.tau_m = tau_m
        
        # Synaptic time constant
        self.tau_s = tau_s
        
        # Spiking threshold
        if T is None:
            self.T = n_syn/4
        else:
            self.T = T
        
        # Model constants
        self.K = K
        self.K1 = K1
        self.K2 = K2

        # The number of synapses
        self.n_syn = n_syn
        
        # The incoming spike times memory window
        if max_spikes is None:
            self.max_spikes = 70
        else:
            self.max_spikes = max_spikes

        # Instantiate a specific tensorflow graph for the Neuron Model
        self.graph = tf.Graph()
        
        ################################
        # Build the neuron model graph #
        ################################
        with self.graph.as_default():

            ##############################
            # Variables and placeholders #
            ##############################    
            self.get_vars_and_ph(w)
            
            ##############
            # Operations #
            ##############
            self.potential = self.get_potential_op()

    ###############################################
    # Define the graph Variables and placeholders #
    ###############################################
    def get_vars_and_ph(self, w):

        # Placeholders (ie things that are passed to the graph as inputs)
        
        # A boolean tensor indicating which synapses have spiked during dt
        self.new_spikes = tf.placeholder(shape=[self.n_syn], dtype=tf.bool, name='new_spikes')

        # The time increment since the last update
        self.dt = tf.placeholder(dtype=tf.float32, name='dt')
        
        # Variables (ie things that are modified by the graph at runtime)

        # The neuron memory of incoming spike times
        self.t_spikes = tf.Variable(tf.constant(100000.0, shape=[self.max_spikes, self.n_syn]), dtype=tf.float32)
        
        # The last spike time insertion index
        self.t_spikes_idx = tf.Variable(self.n_syn - 1, dtype=tf.int32)

        # The relative time since the last spike (assume it was a very long time ago)
        self.last_spike = tf.Variable(1000.0, dtype=tf.float32, name='last_spike')
        
        # The membrane potential
        self.p = tf.Variable(self.p_rest,dtype=tf.float32, name='p')
        
        # The duration remaining in the resting period (between 0 and self.tau_s)
        self.t_rest = tf.Variable(0.0,dtype=tf.float32, name='t_rest')
        
        # The synapse efficacy weights
        self.w = tf.Variable(w)

    # Excitatory post-synaptic potential (EPSP)
    def epsilon_op(self):

        # We only use the negative value of the relative spike times
        spikes_t_op = tf.negative(self.t_spikes)

        return self.K *(tf.exp(spikes_t_op/self.tau_m) - tf.exp(spikes_t_op/self.tau_s))
    
    # Membrane spike response
    def eta_op(self):
        
        # We only use the negative value of the relative time
        t_op = tf.negative(self.last_spike)
        
        # Evaluate the spiking positive pulse
        pos_pulse_op = self.K1 * tf.exp(t_op/self.tau_m)
        
        # Evaluate the negative spike after-potential
        neg_after_op = self.K2 * (tf.exp(t_op/self.tau_m) - tf.exp(t_op/self.tau_s))

        # Evaluate the new post synaptic membrane potential
        return self.T * (pos_pulse_op - neg_after_op)
    
    # Neuron behaviour during integrating phase (t_rest = 0)
    def w_epsilons_op(self):
        
        # Evaluate synaptic EPSPs. We ignore synaptic spikes older than the last neuron spike
        epsilons_op = tf.where(tf.logical_and(self.t_spikes >=0, self.t_spikes < self.last_spike - self.tau_rest),
                               self.epsilon_op(),
                               self.t_spikes*0.0)
                          
        # Agregate weighted incoming EPSPs 
        return tf.reduce_sum(self.w * epsilons_op)

    # Neuron behaviour during resting phase (t_rest > 0)
    def post_firing_p_op(self):
   
        # Membrane potential is only impacted by the last post-synaptic spike (ignore EPSPs)
        return self.eta_op()
    
    def update_spikes_times(self):
        
        # Increase the age of all the existing spikes by dt
        old_spikes_op = self.t_spikes.assign_add(tf.ones(tf.shape(self.t_spikes), dtype=tf.float32) * self.dt)

        # Increment last spike index (modulo max_spikes)
        new_idx_op = self.t_spikes_idx.assign(tf.mod(self.t_spikes_idx + 1, self.max_spikes))

        # Create a list of coordinates to insert the new spikes
        idx_op = tf.constant(1, shape=[self.n_syn], dtype=tf.int32) * new_idx_op
        coord_op = tf.stack([idx_op, tf.range(self.n_syn)], axis=1)

        # Create a vector of new spike times (non-spikes are assigned a very high time)
        new_spikes_op = tf.where(self.new_spikes,
                                 tf.constant(0.0, shape=[self.n_syn]),
                                 tf.constant(100000.0, shape=[self.n_syn]))
        
        # Replace older spikes by new ones
        return tf.scatter_nd_update(old_spikes_op, coord_op, new_spikes_op)
    
    def resting_w_op(self):
        
        # For the base LIF Neuron, the weights remain constants when resting
        return tf.identity(self.w)
    
    def default_w_op(self):
        
        # For the base LIF Neuron, the weights remain constants when integrating
        return tf.identity(self.w)

    def firing_w_op(self):

        # For the base LIF Neuron, the weights remain constants when firing
        return tf.identity(self.w)
    
    def resting_op(self):
        
        # Update weights
        w_op = self.resting_w_op()
        
        # Update the resting period
        t_rest_op = self.t_rest.assign(tf.maximum(self.t_rest - self.dt, 0.0))
        
        # During the resting period, the membrane potential is only given by the eta kernel
        with tf.control_dependencies([w_op, t_rest_op]):
            return self.eta_op()
    
    def firing_op(self):

        # Update weights
        w_op = self.firing_w_op()
        
        # Reset the time of the last spike, but only once the weights have been updated
        with tf.control_dependencies([w_op]):
            last_spike_op = self.last_spike.assign(0.0)

        # Start the resting period
        t_rest_op = self.t_rest.assign(self.tau_rest)
        
        # At spiking time, the membrane potential is only given by the eta kernel
        with tf.control_dependencies([last_spike_op, t_rest_op]):
            return self.eta_op()
        
    def default_op(self):
        
        # Update weights
        w_op = self.default_w_op()
        
        # By default, the membrane potential is given by the sum of the eta kernel and the weighted epsilons
        with tf.control_dependencies([w_op]):
            return self.eta_op() + self.w_epsilons_op()
        
    def integrating_op(self):

        # Evaluate the new membrane potential, integrating both synaptic input and spike dynamics
        p_op = self.eta_op() + self.w_epsilons_op()

        # We have a different behavior if we reached the threshold
        return tf.cond(p_op > self.T,
                       self.firing_op,
                       self.default_op)
    
    def get_potential_op(self):
        
        # Update our internal memory of the synapse spikes (age older spikes, add new ones)
        update_spikes_op = self.update_spikes_times()
        
        # Increase the relative time of the last spike by the time elapsed
        last_spike_age_op = self.last_spike.assign_add(self.dt)
        
        # Update the internal state of the neuron and evaluate membrane potential
        with tf.control_dependencies([update_spikes_op, last_spike_age_op]):
            return tf.cond(self.t_rest > 0.0,
                           self.resting_op,
                           self.integrating_op)

### A class that generates random spike trains

In [57]:
class SpikeTrains(object):
    
    def __init__(self, n_syn, r_min=0.0, r_max=90.0, r=None, s_max=1800, ds_max=360, s=None, auto_vrate=True, delta_max=0):
        
        # Number of synapses
        self.n_syn = n_syn
        # Minimum and maximum spiking rate (in Hz)
        self.r_min = r_min
        self.r_max = r_max
        # Spiking rate for each synapse (in Hz)
        if r is None:
            self.r = np.random.uniform(self.r_min, self.r_max, size=(n_syn))
        else:
            self.r = r
        # Rate variation parameters
        self.s_max = s_max
        self.ds_max = ds_max
        # Rate variation
        if s is None:
            self.s = np.random.uniform(-self.s_max, self.s_max, size=(self.n_syn))
        else:
            self.s = s
        # Automatically apply rate variation when
        self.auto_vrate = auto_vrate
        # Maximum time between two spikes on each synapse (0 means no maximum) in ms
        self.delta_max = delta_max

        # Memory of spikes
        self.spikes = None
    
    # Generate new spikes for the specified time interval (in ms)
    # The new spikes are added to the existing spike trains.
    # The method returns only the new set of spikes
    def add_spikes(self, t):
        
        for step in range(t):
            # Draw a random number for each synapse
            x = np.random.uniform(0,1, size=(self.n_syn))
            # Each synapse spikes if the drawn number is lower than the probablity
            # given by the integration of the rate over one millisecond
            spikes = x < self.r * 1e-3
            # Keep a memory of our spikes
            if self.spikes is None:
                self.spikes = np.array([spikes])
            else:
                if self.delta_max > 0:
                    # We force each synapse to spike at least every delta_max ms
                    if self.spikes.shape[0] < self.delta_max - 1:
                        # At the beginning of the trains, we try to 'fill' as much holes
                        # as possible to avoid a 'wall of spikes' when we reach delta_max.
                        # For each synapse, count non-zero items
                        n_spikes = np.count_nonzero(self.spikes, axis=0)
                        # Draw a random number for each synapse 
                        r = np.random.uniform(0.0, 1.0, size=self.n_syn)
                        # The closer we get to delta_max, the higher probability we have to force a spike
                        forced_spikes = r < step * 1.0/self.delta_max
                        # Modify our random vector of spikes for synapse that did not spike
                        spikes = np.where(n_spikes > 0, spikes, spikes | forced_spikes)
                    else:
                        # Get the last delta_max -1 spike trains
                        last_spikes = self.spikes[-(self.delta_max - 1):,:]
                        # For each synapse, count non-zero items
                        n_spikes = np.count_nonzero(last_spikes, axis=0)
                        # Modify spikes to force a spike on synapses where the spike count is zero
                        spikes = np.where(n_spikes > 0, spikes, True)
                # Store spikes
                self.spikes = np.append(self.spikes, [spikes], axis=0)
            if self.auto_vrate:
                self.change_rate()

        return self.spikes[-t:,:]
    
    # Format a list of spike indexes
    def get_spikes(self):
        
        real_spikes = np.argwhere(self.spikes > 0)
        # We prefer having spikes in the range [1..n_syn]
        spike_index = real_spikes[:,1] + 1
        spike_timings = real_spikes[:,0]
        
        return spike_timings, spike_index
    
    # Change rate, applying the specified delta in Hz
    def change_rate(self, delta=None):

        # Update spiking rate
        if delta is None:
            delta = self.s
        self.r = np.clip( self.r + delta, self.r_min, self.r_max)
        # Update spiking rate variation
        ds = np.random.uniform(-self.ds_max, self.ds_max, size=(self.n_syn))
        self.s = np.clip( self.s + ds, -self.s_max, self.s_max)

## Code for Spatio Temporal Classification using SNN

### Generating Spike Trains

In [71]:
np.random.seed(123)
spike_times = []
for i in range(0,5):
    s = np.random.randint(low=1,high=199,size=200)
    spike_times.append(s)

spike_train = []
for i in range(0,5):
    s = np.zeros((200,200),dtype = bool)
    spike_train.append(s)

for i in range(0,5):
    for j in range(0,200):
        k = spike_times[i][j]
        spike_train[i][k][j] = True

# Desired Spike Times
desired_spike_times_list = []
desired = [33,66,99,132,165]
for i in range(0,5):
    desired_spike_times_list.append(desired[i])

### Classification Task using SNN

In [73]:
def snn(weights):
    T = 200
    # Duration of each time step in ms
    dt = 1.0
    # Number of iterations = T/dt
    steps = int(T / dt)
    # Number of synapses
    n_syn = 200
    # Our random spike trains
    spike_trains = SpikeTrains(n_syn)
    # Generate spikes over the specified period
    syn_has_spiked = spike_trains.add_spikes(T)
    #Generate boolean spike trains corresponding to the above spike times
    syn_has_spiked = spike_train 

    # We define the base synaptic efficacy as a uniform vector
    # Output variables
    P = []
    W = np.copy(weights)
    # Instantiate our LIF neuron
    neuron = LIFNeuron(n_syn, W,T=20)
    actual_spike_times_list = []
    
    with tf.Session(graph=neuron.graph) as sess:

        sess.run(tf.global_variables_initializer())
    
        for batch in range(0,5):
            
            actual_spike_times = []
            
            for step in range(steps):
                t = step * dt
                feed = { neuron.new_spikes: syn_has_spiked[batch][step], neuron.dt: dt}
                p = sess.run(neuron.potential, feed_dict=feed)
                P.append((t,p))
                if(p>20):
                    actual_spike_times.append(step)    
            
            actual_spike_times_list.append(actual_spike_times)
    return(actual_spike_times_list)

## Evolutionary Algorithm 1 - Real Coded Genetic Algorithms

### Fitness Function

In [217]:
def fitness_function(desired_times,population):
    diff = []
    diff_list = []
    s = []
    l = 0
    actual_spike_times_list = []
    acc_list = []
    
    for i in range(0,len(population)):
        actual_spike_times_list.append(snn(population[i]))
    
    for i in range(0,len(actual_spike_times_list)):
        for j in range(0,len(actual_spike_times_list[i])):
            if(len(actual_spike_times_list[i][j])!=0):
                for k in range(0,len(actual_spike_times_list[i][j])):
                    s.append(actual_spike_times_list[i][j][k] - desired_times[l])
            else:
                    s.append(0-desired_times[l])
            l= l + 1
            diff.append(np.mean(np.absolute(s)))
            s = []
        l=0
        diff_list = np.array(diff)
        diff = []
        err = np.where(diff_list>=3)
        acc = 100 - (len(err[0])/len(diff_list))*100
        acc_list.append(acc)

    return(acc_list)

In [98]:
population = []
for i in range(0,5):
    s = np.float32(np.random.uniform(low=0,high=25,size=200))
    population.append(s)

fitness_function([33,66,99,132,165],population)

5
[75.46666666666667, 52.354838709677416, 53.15625, 54.44827586206897, 71.06666666666666]
(array([0, 1, 2, 3, 4], dtype=int64),)
[74.78125, 54.166666666666664, 53.1, 55.56666666666667, 73.3030303030303]
(array([0, 1, 2, 3, 4], dtype=int64),)
[77.06451612903226, 56.225806451612904, 52.9, 56.516129032258064, 71.56666666666666]
(array([0, 1, 2, 3, 4], dtype=int64),)
[74.96428571428571, 52.407407407407405, 53.096774193548384, 54.70967741935484, 68.73333333333333]
(array([0, 1, 2, 3, 4], dtype=int64),)
[72.58620689655173, 50.42857142857143, 54.42857142857143, 54.064516129032256, 76.06451612903226]
(array([0, 1, 2, 3, 4], dtype=int64),)


[0.0, 0.0, 0.0, 0.0, 0.0]

In [99]:
def crossover(p1,p2):
    h1 = 0.5*p1 + 0.5*p2
    h2 = 1.5*p1 - 0.5*p2
    h3 = 1.5*p2 - 0.5*p1
    return(h1,h2,h3)

In [174]:
def mutation(solution,max_solution,min_solution,current_gen,max_gen):
    rand = np.random.uniform(0,1)
    if(rand<=0.5):
        tau = 1
    else:
        tau = -1
    b = 1
    d = math.pow((1 - (current_gen/max_gen)),b)
    new_solution = solution + tau*(max_solution - min_solution)*(1-math.pow(rand,d))
    return(new_solution)

In [191]:
def genetic_algorithm(population,desired_times,num_epochs):
    
    for epochs in range(0,num_epochs):
    # Compute Fitness Value of entire population and store it in a list
    
        fitness = fitness_function(desired_times,population)

        fitness_copy = fitness

        indexes = np.argsort(-np.array(fitness))
        fitness = -np.sort(-np.array(fitness))

        index = indexes[0:2]

        # Compare fitness values of population and pick best 2 parents

        parent_1 = population[index[0]]
        parent_2 = population[index[1]]

        # Do crossover to get 2 children 

        child_1,child_2,child_3 = crossover(parent_1,parent_2)

        # Decide whether to add them to population based on their fitness

        children = []
        children.extend([child_1,child_2,child_3])
        population.append(child_1)
        population.append(child_2)
        population.append(child_3)

        fitness_copy.extend(fitness_function(desired_times,children))

        index = np.argsort(-np.array(fitness_copy))
        fitness_copy = -np.sort(-np.array(fitness_copy))

        index_remove = index[len(index)-4:len(index)-1]
        fitness_copy = [i for j, i in enumerate(fitness_copy) if j not in index_remove]
        population = [i for j, i in enumerate(population) if j not in index_remove]
    
        # Sort again to get indexes of best and worst solution get indexes
        
        index = np.argsort(-np.array(fitness_copy))
        best_solution = population[index[0]]
        worst_solution = population[index[len(index)-1]]
        
        # Do mutation
        
        for i in range(0,len(population)):
            population[i] = mutation(population[i],best_solution,worst_solution,epochs,num_epochs)
            population[i][population[i]>15] = 15
            population[i][population[i]<-15] = -15
        
        fitness_copy = fitness_function(desired_times,population)

    return(fitness_copy,population)
    
    # End GA

In [221]:
##### Test Here!!
T = 200
# Duration of each time step in ms
dt = 1.0
# Number of iterations = T/dt
steps = int(T / dt)
# Number of synapses
n_syn = 200
# Our random spike trains
spike_trains = SpikeTrains(n_syn)
# Generate spikes over the specified period
syn_has_spiked = spike_trains.add_spikes(T)
#Generate boolean spike trains corresponding to the above spike times
syn_has_spiked = spike_train 
neuron = LIFNeuron(n_syn, weights[48],T=20)
actual_spike_times_list = []
P = []
with tf.Session(graph=neuron.graph) as sess:

    sess.run(tf.global_variables_initializer())
    
    for batch in range(0,5):
            
        actual_spike_times = []
            
        for step in range(steps):
            t = step * dt
            feed = { neuron.new_spikes: syn_has_spiked[batch][step], neuron.dt: dt}
            p = sess.run(neuron.potential, feed_dict=feed)
            P.append((t,p))
            if(p>20):
                actual_spike_times.append(step)    
            
        actual_spike_times_list.append(actual_spike_times)
print(actual_spike_times_list)

[[68, 122, 141], [49, 84, 191], [99], [2, 21, 103, 125, 132], [3, 40, 159, 170]]


In [218]:
population = []
for i in range(0,50):
    s = np.float32(np.random.uniform(low=-15,high=15,size=200))
    population.append(s)

fit,weights = genetic_algorithm(population,[33,66,99,132,165],5)

In [219]:
print(fit)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 0.0, 0.0, 0.0, 20.0, 0.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]


## Evolutionary Algorithm 2 - Differential Evolution

In [ ]:
def de(fobj, bounds, mut=0.8, crossp=0.7, popsize=20, its=1000): # fobj can be a list of fitness values, bounds on values
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
        
        yield best, fitness[best_idx]

## Evolutionary Algorithm 3 - Particle Swarm Optimization

In [ ]:
def pso_initialize(pop_size):
    pos = []
    vel = []
    for i in range(0,pop_size):
        s = np.random.uniform(0,)